In [30]:
!pip install datasets

# 1. Tokenization (15 pts)

How does one represent textual input to language models? One strategy that we have seen is to split up words on spaces, e.g.,

> This is an example.

> [This, is, an, example],

but this fails when unseen words appear at test time, e.g.,

> We named our son nwonkun.

> [We, named, our, son, \<unk\>] (5 tokens).

One solution to this problem is to use character-level tokens

> [W, e, _, n, a, m, e, d, _, o, u, r, _, s, o, n, _, n, w, o, n, k, u, n]

(24 tokens, if I counted right), but now the number of tokens required to encode a sentence has increased a *lot*.

## 1.1 Byte-pair encodings and sub-word tokenization (5 pts)

[Byte-pair encodings (BPE)](https://en.wikipedia.org/wiki/Byte_pair_encoding) are a clever middle ground for the tokenization problem.
Starting from a character-level tokenization, iteratively combine the most common bigrams (token pairs) into their own tokens.
For example, the most common bigrams from the previous example are "_n" and "on". Breaking the tie arbitrarily and creating a new token "_n" we now have

> [W, e, _n, a, m, e, d, _, o, u, r, _, s, o, n, _n, w, o, n, k, u, n]

reducing the token count to 22. Iteratively applying this rule, we can further reduce it to 20 tokens by adding the token "on", and so on. Each step of this algorithm greedily reduces the token count by the maximum amount.

This tokenization scheme, known as "sub-word tokenization" takes the best of both worlds: since the vocabulary still contains tokens for every byte, we never have to use the \<unk\> token, while still reducing the number of required tokens to encode a sequence. The more tokens you add, the shorter your sequence gets.

To decide which tokens to add to the vocabulary, we have to *train* our BPE tokenizer on a corpus.
In this section you will do just that.

In [31]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
train: str = str.join(" ", dataset["train"]["text"])[:pow(10, 6)]
test: str = str.join(" ", dataset["test"]["text"])[:pow(10, 6)]

In [32]:
from itertools import chain, pairwise
from collections import Counter
from tqdm import tqdm

class Tokenizer:
    # The lookup list contains *byte groups*, represented as a tuples of ints.
    # The token ID for a byte group is its index in the list.
    vocab: list[tuple[int, ...]]

    def __init__(self, training_seq: str, vocab_size: int) -> None:
        # Initialize a lookup with single-byte groups
        self.vocab = [(i,) for i in range(pow(2, 8))]
        byte_seq = list(bytes(training_seq, "utf-8"))
        for i in tqdm(range(pow(2, 8), vocab_size)):
            """
            TODO: iteratively add the most common token pairs to the vocabulary.
            Advice: try using Counter and pairwise from the python std lib.
            """
            token_seq = self.tokenize(training_seq)
            pair_counts = Counter(tuple(pair) for pair in pairwise(token_seq))
            if not pair_counts:
                break
            most_common = max(pair_counts.items(), key=lambda x: x[1])[0]
            new_token = tuple(chain.from_iterable(self.vocab[t] for t in most_common))
            self.vocab.append(new_token)

    def tokenize(self, seq: str) -> list[int]:
        """
        TODO: convert a byte sequence into a token sequence by greedily adding
        the longest token that matches the rest of the sequence, e.g.,
        vocab = [a, aa, b]
        sequence = aaab
        token_seq = [1, 0, 2] NOT [0, 1, 2].
        """
        byte_seq: list[int] = list(bytes(seq, "utf-8"))
        token_seq = []
        i = 0
        while i < len(byte_seq):
            best_len = 1
            best_token = self.vocab.index((byte_seq[i],))
            for token_id, token in enumerate(self.vocab):
                token_len = len(token)
                if (i + token_len <= len(byte_seq) and
                    tuple(byte_seq[i:i + token_len]) == token and
                    token_len > best_len):
                    best_len = token_len
                    best_token = token_id
            token_seq.append(best_token)
            i += best_len
        return token_seq

    def detokenize(self, token_seq: list[int]) -> str:
        # TODO: convert a token sequence into a byte sequence.
        byte_seq = list(chain.from_iterable(self.vocab[t] for t in token_seq))
        return bytes(byte_seq).decode("utf-8")


train_data = train[:10000]
tokenizer = Tokenizer(train_data, vocab_size=500)

print("Some of our new tokens:")
for token in tokenizer.vocab[-10:]:
    print(repr(bytes(token).decode("utf-8")))

100%|██████████| 244/244 [04:00<00:00,  1.02it/s]

Some of our new tokens:
'squad '
'ha'
'her '
'batt'
'wea'
'Ar'
'war'
'my '
'der '
'Jap'


As a sanity check, your implementation should be able to compress the training set to ~40-50% of its original size.
You should notice that the test set compression does not perform as well. This is because the distribution of bigrams in the test set does not exactly match the that of the train set. This gets worse the further your test set distribution is from your training set.

In [33]:
# Do not edit this code cell
test_data = test[:10000]
train_bytes_len = len(bytes(train_data, "utf-8"))
train_token_len = len(tokenizer.tokenize(train_data))
print(f"Compressed train set to {train_token_len / train_bytes_len * 100:.0f}% original size")
test_bytes_len = len(bytes(test_data, "utf-8"))
test_token_len = len(tokenizer.tokenize(test_data))
print(f"Compressed test set to {test_token_len / test_bytes_len * 100:.0f}% original size")

assert train_data == tokenizer.detokenize(tokenizer.tokenize(train_data))
assert test_data == tokenizer.detokenize(tokenizer.tokenize(test_data))

Compressed train set to 41% original size
Compressed test set to 52% original size


## 1.2 BPE performance on OOD text. (5 pts)

Explore how English-trained BPE performs on non-English text by downloading corpora from a few different languages and using your English-trained tokenizer. What do you find? Do the results match your expectations? For what langauges does the tokenizer struggle with the most? How might this impact society if everyone were to use your tokenizer?

Include your code, results, and discussion in new cells below.

Hint: we recommend you use `load_dataset` to fetch from HuggingFace with `streaming=True` to avoid huge downloads. You might want to take a look at the `oscar` dataset.

In [34]:
from datasets import load_dataset
from collections import defaultdict
from tqdm import tqdm

langs = ["es", "fr", "de", "zh"]  # Spanish, French, German, Chinese
res = defaultdict(list)

for lang in langs:
    ds = load_dataset("oscar", f"unshuffled_deduplicated_{lang}", split="train", streaming=True)
    sample_texts = [text["text"] for text in ds.take(100)]
    ttl = 0
    for sentence in sample_texts:
        tokenized = tokenizer.tokenize(sentence)
        ttl += len(tokenized)
    avg_token_length = ttl / len(sample_texts)
    res[lang] = avg_token_length
    print(f"Average token length for {lang} (sample of 100 sentences): {avg_token_length}")

print("Results:", dict(res))


Average token length for es (sample of 100 sentences): 2370.19
Average token length for fr (sample of 100 sentences): 1397.1
Average token length for de (sample of 100 sentences): 2153.35
Average token length for zh (sample of 100 sentences): 16104.22
Results: {'es': 2370.19, 'fr': 1397.1, 'de': 2153.35, 'zh': 16104.22}



In my opinion, the English-trained tokenizer runs pretty well on languages similar to English (e.g. French, Spanish), it suffers a lot on structurally different ones including Chinese. You can tell from the extremely high token counts for chinese, it breaks sentences into many tiny meaningless tokens.

I expected these results as the tokenizer is trained on English it seems to perform better on similar script languages. The high token count for Chinese was expected based on its unique script and structure, whereas German continues to be a challenge due mainly to its compound words.

I think if this tokenizer was employed for all the languages globally, it would result in inefficiencies and higher processing time + computational costs for non-Latin scripts such as Chinese. It could also instill biases, because the system will optimize for languages similar to English and ultimately limit or misrepresent speakers of other languages. And at the same time, for inclusive AI, we think a more flexible tokenizer is necessary.


## 1.3 Pitfalls of and alternatives to BPE (5 pts)

BPE tokenization sufferes from other issues as well. Due to the implementation of our BPE tokenizer, detokenizing a sequence of tokens then re-tokenizing it does not always recover the original sequence:
```
vocab = {a, aa, b}
tokens = [0, 1, 2]
detokenized = aaab
retokenized = [1, 0, 2]
```

Another issue is that some tokens that may have been prevalent during BPE training may not be present during language model training, leading to funky situations where the language model has not been trained to represent or output some tokens. See this paper for more information: https://arxiv.org/pdf/2405.05417.

Some NLP researchers think that we should move away from sub-word tokenization to get rid of these problems. Engage with this discussion by either
- Finding a paper that points out an issue with tokenization and propose your own solution for how you would fix it, or
- Finding a paper that proposes an alternative tokenization scheme (or way of processing text) and discuss the drawbacks of the proposed method.

Your response should be about a paragraph in length and link to a paper.

I went ahead with  *Finding a paper that proposes an alternative tokenization scheme (or way of processing text) and discuss the drawbacks of the proposed method.*

I found a paper named "Subword Regularization: Improving Neural Network Translation Models
with Multiple Subword Candidates" which proposes a probabilistic tokenization method that competes against the popular byte-pair encoding (BPE). Rather than merging the most common character pairs, it starts off with a huge vocabulary of subwords and does top down through the detail which retains likely candidates based on a uni-gram model whilst pruning others. This approach is relatively more flexible towards different data, and can easily accommodate out-of-vocabulary words.

It has the following disadvantages: it is computationally expensive because of probabilistic calculations involved, it is lacking with inflectionally rich languages and humans can also find the segmentations meaningless on unseen words due to potentially sub-optimal segmentations leading to tokens that are less interpretable.

Link to the paper: https://arxiv.org/abs/1804.10959

# 2. Generation Algorithms (35 pts + 15 pts BONUS)

In this problem, we will implement several common decoding algorithms and test them with the GPT-2 Medium model.

Given the class below, we will fill in each of the method stubs. You may create additional helper methods as well to make components re-usable.

**You are not allowed to use the generate() function in the transformers library. You can only use the model's forward() method to retrieve final layer logits**

In addition to the methods we ask you to implement, which are:
- Greedy decoding
- Temperature Sampling
- Nucleus Sampling

You will choose ONE of the following sampling algorithms to implement as well (make sure to add your own method, since we do not provide one by default):
- Typical Sampling ([Meister et al. (2022)](https://arxiv.org/abs/2202.00666))
- Eta Sampling ([Hewitt et al. (2022)](https://arxiv.org/abs/2210.15191))

Points for this question will be distributed as follows:

- 5-10 points for implementing each decoding algorithm
- 5 points for implementing the generate() function (you will make this incrementally through each sub-part)
- 5 points for filling out the table with list of tokens (see instructions below)

In [35]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import Optional

class LM():
  def __init__(self, model_name: str = "openai-community/gpt2-medium"):
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.model = AutoModelForCausalLM.from_pretrained(model_name)
    self.model.eval()

  def greedy_decoding(self, prompt: str, max_length: int = 64) -> str:
    """
    TODO:

    Implement greedy decoding, in which we use the highest
    probability token at each decoding step
    """
    inp_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
    otp_ids = []

    with torch.no_grad():
        for _ in range(max_length):
            otp = self.model(input_ids=inp_ids)
            logits = otp.logits[:, -1, :]
            nxt_tkn_id = torch.argmax(logits, dim=-1)
            otp_ids.append(nxt_tkn_id.item())
            inp_ids = torch.cat([inp_ids, nxt_tkn_id.unsqueeze(0)], dim=-1)

            if nxt_tkn_id.item() == self.tokenizer.eos_token_id:
                break

    return self.tokenizer.decode(otp_ids, skip_special_tokens=True)

  def temperature_sampling(self, prompt: str, temperature: float = 1.0, max_length: int = 64) -> str:
    """
    TODO:

    Implement temperature sampling, in which we sample
    from the output distribution at each decoding step,
    with a temperature parameter to control the "peakiness"
    of the output distribution
    """
    inp_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
    otp_ids = inp_ids[0].tolist()

    with torch.no_grad():
        for _ in range(max_length):
            otp = self.model(input_ids=inp_ids)
            logits = otp.logits[:, -1, :]

            if temperature == 0:
                nxt_tkn_id = torch.argmax(logits, dim=-1, keepdim=True)
            else:
                logits = logits / temperature
                probs = torch.softmax(logits, dim=-1)
                nxt_tkn_id = torch.multinomial(probs, num_samples=1)

            otp_ids.append(nxt_tkn_id.item())
            inp_ids = torch.cat([inp_ids, nxt_tkn_id], dim=-1)

            if nxt_tkn_id.item() == self.tokenizer.eos_token_id:
                break

    return self.tokenizer.decode(otp_ids, skip_special_tokens=True)

  def nucleus_sampling(self, prompt: str, p: float = 0.9, max_length: int = 64, temperature: float = 1.0) -> str:
    """
    TODO:
    Implement nucleus sampling, in which we
    sample from a subset of the vocabulary
    at each decoding step
    Note: There is also a temperature parameter here
    """
    inp_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
    otp_ids = inp_ids[0].tolist()

    for _ in range(max_length):
        otp = self.model(input_ids=inp_ids)
        logits = otp.logits[:, -1, :] / temperature
        probs = torch.softmax(logits, dim=-1)
        srtd_probs, srtd_idx = torch.sort(probs, descending=True)
        cm_probs = torch.cumsum(srtd_probs, dim=-1)
        cm_mask = cm_probs <= p
        cm_mask[..., 1:] = cm_mask[..., :-1].clone()
        cm_mask[..., 0] = True
        top_p_probs = srtd_probs[cm_mask]
        top_p_idx = srtd_idx[cm_mask]
        nxt_tkn_idx = torch.multinomial(top_p_probs, num_samples=1).item()
        nxt_tkn_id = top_p_idx[nxt_tkn_idx].item()
        otp_ids.append(nxt_tkn_id)
        inp_ids = torch.cat([inp_ids, torch.tensor([[nxt_tkn_id]])], dim=-1)

        if nxt_tkn_id == self.tokenizer.eos_token_id:
            break

    return self.tokenizer.decode(otp_ids, skip_special_tokens=True)

  def typical_sampling(self, prompt: str, eta: float = 0.2, max_length: int = 64) -> str:
    """
    TODO:

    Implement typical sampling, which focuses on sampling
    from tokens that are within a certain "surprise" range
    based on entropy to prevent unlikely tokens from being chosen.
    """
    inp_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
    otp_ids = inp_ids[0].tolist()

    for _ in range(max_length):
        logits = self.model(input_ids=inp_ids).logits[:, -1, :]
        probs = torch.softmax(logits, dim=-1)
        srtd_probs, srtd_idx = torch.sort(probs, descending=True)
        cm_probs = torch.cumsum(srtd_probs, dim=-1)
        typical_mask = cm_probs <= eta
        typical_probs = srtd_probs[typical_mask]
        typical_idx = srtd_idx[typical_mask]

        if typical_probs.numel() == 0:
            nxt_tkn_id = srtd_idx.squeeze()[0].item()
        else:
            nxt_tkn_id = typical_idx[torch.multinomial(typical_probs, num_samples=1)].item()

        otp_ids.append(nxt_tkn_id)
        inp_ids = torch.cat([inp_ids, torch.tensor([[nxt_tkn_id]])], dim=-1)

        if nxt_tkn_id == self.tokenizer.eos_token_id:
            break

    return self.tokenizer.decode(otp_ids, skip_special_tokens=True)


  def generate(self,
               prompt: str,
               temperature: float = 1.0,
               p: Optional[float] = None) -> str:
      """
      TODO:

      Route to the appropriate generation function
      based on the arguments
      HINT: What parameter values should map to greedy decoding?
      """
      if p is None and temperature == 1.0:
        return self.greedy_decoding(prompt)
      elif p is not None:
        return self.nucleus_sampling(prompt, p=p, temperature=temperature)
      else:
        return self.temperature_sampling(prompt, temperature=temperature)


In [36]:
GPT2LM = LM("openai-community/gpt2-medium")

For each sampling algorithm you implement, fill out this table, in which you will list the top 10 highest probability tokens **at the first decoding step** in a comma separated list. For algorithms like nucleus sampling where you perform some kind of truncation/re-distribution of the output distribution, do the truncation/re-distribution first, and then sort the vocabulary by probability to complete the table.

For this and all questions below, use the following prompt:


**"Once upon a time in a land far far away, "**

Note: Use the default value for `max_length` for all questions below.

In [37]:
prompt = "Once upon a time in a land far far away, "
GPT2LM = LM(model_name="openai-community/gpt2-medium")

print("Greedy Decoding Top 10 Tokens:", GPT2LM.greedy_decoding(prompt, max_length=10))
print("Temperature Sampling Top 10 Tokens (t=1.0):", GPT2LM.temperature_sampling(prompt, temperature=1.0, max_length=10))
print("Nucleus Sampling Top 10 Tokens (p=0.9):", GPT2LM.nucleus_sampling(prompt, p=0.9, temperature=1.0, max_length=10))
print("Typical Sampling Top 10 Tokens (eta=0.2):", GPT2LM.typical_sampling(prompt, eta=0.2, max_length=10))


Greedy Decoding Top 10 Tokens:  a young man named   was born
Temperature Sampling Top 10 Tokens (t=1.0): Once upon a time in a land far far away, ・ Folklorist (C)

──
Nucleus Sampling Top 10 Tokens (p=0.9): Once upon a time in a land far far away,  Bonds established an outfit which was really scary
Typical Sampling Top 10 Tokens (eta=0.2): Once upon a time in a land far far away,  a man named Tiamat came to the


| **Decoding Algorithm** | **10 Highest Probability Tokens** |
|------------------------|-----------------------------------|
| Greedy                 |   a young man named   was born                                 |
| Temperature (t=1.0)    | Once upon a time in a land far far away, ・ Folklorist (C)                                 |
| Nucleus (p=0.9)        | Once upon a time in a land far far away,  Bonds established an outfit which was really scary                                 |
| Typical/Eta            | Once upon a time in a land far far away,  a man named Tiamat came to the                                 |

## 2.1 Greedy Decoding (5 points)

First, implement the most simple decoding method of greedy decoding. Here, at each decoding time step, simply use the highest probability token. Note that you'll need to adjust the generate function so that a specific temperature value will map to greedy decoding (what should that value be?).

Use the prompt given above to test your implementation. What do you notice?

In [38]:
prompt = "Once upon a time in a land far far away,"
GPT2LM = LM()

print("Greedy Decoding Output:", GPT2LM.generate(prompt, temperature=0))

Greedy Decoding Output: Once upon a time in a land far far away, there lived a man named Simeon. He was a wise man, and he knew the secrets of the universe. He knew that the universe was made of many worlds, and that each world was made of a single substance. He knew that each substance was made of a single substance, and that each substance was made


I noticed that by setting the temperature = 0, causes the model to choose the highest probability token at every step, which leads to sensible outputs. However, greedy decoding may produce constant or non-surprising answers since it makes the model choose only top options. This method on the other hand preserves consistency in terms of quality.

##2.2 Temperature Sampling (10 pts)

Sometimes (a lot of the time?), we don't actually just want the highest probability token at each time step. Why might this be the case?

Unrestrained by always picking the token with the maximum probability, we breathe diversity and creativity into Temperature Sampling. It attempts to avoid generating repetitive or overly tepid text, by drawing from a distribution of plausible tokens. The temperature parameter controls this randomness; a larger temperature will increase diversity and creativity in the outputs, whereas lower temperatures lead to more concentrated and deterministic results. Its tradeoff between coherence and diversity makes temperature sampling a perfect fit for producing provocative and less deterministic text.

To adjust for this, we often use sampling algorithms instead of greedy decoding. However, there are many ways we can go about sampling.

First, implement temperature sampling. Recall that the temperature parameter adjusts the "randomness" of the output at each time step. Here, you'll need to think about how to adjust the output distribution which you will do multinomial sampling from. Be careful about how you will handle very low (close to 0) temperatures.

Given the same prompt as above, test your implementation with the following temperature values: [0.3, 0.5, 0.7, 0.9, 1.1]. For each value, sample 3 outputs. What do you notice in terms of the differences between output sets across different temperature values?  

In [39]:
prompt = "Once upon a time in a land far far away,"
GPT2LM = LM()
temp_vals = [0.3, 0.5, 0.7, 0.9, 1.1]

for temp in temp_vals:
    print(f"\nTemperature: {temp}")
    for i in range(3):
        otp = GPT2LM.temperature_sampling(prompt, temperature=temp, max_length=10)
        print(f"Output {i+1}: {otp}")



Temperature: 0.3
Output 1: Once upon a time in a land far far away, there lived a king who was fond of many things
Output 2: Once upon a time in a land far far away, there was a young man named Sibyl.
Output 3: Once upon a time in a land far far away, a young man named Emmett Brown was born

Temperature: 0.5
Output 1: Once upon a time in a land far far away, a young man was born and raised in a small
Output 2: Once upon a time in a land far far away, a princess was born. A princess, who loved
Output 3: Once upon a time in a land far far away, there lived a king who lived by his own laws

Temperature: 0.7
Output 1: Once upon a time in a land far far away, a young warrior named Gob was entrusted with the duty
Output 2: Once upon a time in a land far far away, there was a young girl named Lona. She
Output 3: Once upon a time in a land far far away, where the sun never sets, and the moon never

Temperature: 0.9
Output 1: Once upon a time in a land far far away, rumor and legend would blur

| **Temperature** | **Output 1** | **Output 2** | **Output 3** |
|-----------------|--------------|--------------|--------------|
| 0.3             | Once upon a time in a land far far away, there lived a king who was fond of many things            |Once upon a time in a land far far away, there was a young man named Sibyl.            |Once upon a time in a land far far away, a young man named Emmett Brown was born            |
| 0.5             | Once upon a time in a land far far away, a young man was born and raised in a small            |Once upon a time in a land far far away, a princess was born. A princess, who loved            |Once upon a time in a land far far away, there lived a king who lived by his own laws  |
| 0.7             | Once upon a time in a land far far away, a young warrior named Gob was entrusted with the duty            | Once upon a time in a land far far away, there was a young girl named Lona. She            | Once upon a time in a land far far away, where the sun never sets, and the moon never|
| 0.9             |Once upon a time in a land far far away, rumor and legend would blur and fail to exist.|Once upon a time in a land far far away, in an era far, far apart, old Scout |Once upon a time in a land far far away, I observed my two strongest brothers, who were young            |
| 1.1             |Once upon a time in a land far far away, a young princess traveled far and wide with an enchanted            |Once upon a time in a land far far away, valiant Americans touched down here on the islands of O            |Once upon a time in a land far far away, the tale was told. It was tucked away under            |

I noticed the following differences between output sets across different temperature values:

1. Low Temp (0.3, 0.5) — Tend to be more predictable and consistent by following simple patterns

2. Middle Temp (0.7): Creativity with balance in coherence — adding a unique piece of information

3. High Temp (0.9, 1.1): Best if it should be the highly diverse but more scattered best for random exploration

In simple terms, higher temp gives more creative outputs.

## 2.3 Nucleus Sampling (10 pts)

Originally published in [Holtzmann et al. (2021)](https://arxiv.org/abs/1904.09751), nucleus sampling was designed to address an issue that was especially prevalent in language models at the time.

This issue is the case of "neural text degeneration," where outputs from LMs would often degenerate into gibberish if a low probability token was ever decoded. To address this, nucleus (also known as top-p) sampling uses a hyperparameter, p, to control how big of a subset of the vocabulary we sample from at each step. For example, if p=0.9, we only sample from the subset of tokens that have a cumulative probability mass of 0.9 (after sorting by probability).

Implement nucleus sampling and then use the same prompt as above and test your implementation with the following p-values: [0.97, 0.95, 0.9, 0.8, 0.7]
What do you notice across outputs?

In [40]:
prompt = "Once upon a time in a land far far away, "
GPT2LM = LM(model_name="openai-community/gpt2-medium")

def test_nucleus_sampling(prompt, p_values):
    for p in p_values:
        print(f"\nNucleus Sampling (p={p})")
        for i in range(1, 4):
            print(f"Output {i}: {GPT2LM.nucleus_sampling(prompt, p=p, max_length=50)}")
p_values = [0.97, 0.95, 0.9, 0.8, 0.7]
test_nucleus_sampling(prompt, p_values)


Nucleus Sampling (p=0.97)
Output 1: Once upon a time in a land far far away, Ȓu I was lucky enough to witness the keys of creation convey the primal spirits and truth of creation. Ȓu was reminded of the absurdity of such crude stargazing when I became contaminated with their false lore about humans, �
Output 2: Once upon a time in a land far far away, _____ (seventh ed.), written by Phisate of Navias, but scanned and posted by Daniel Schwartz
It has been noticed (March 2008) that every 'Title of Person of Notary' either seems excessively long, hesitates to mention
Output 3: Once upon a time in a land far far away,  Campion's predecessor enjoyed a number of sons and daughters married to Clements Hold El. Learning of their fabulations, Clan Evangeline struck ever harder at their kin - forcing them to shrink and adding even more delights to their nursery,

Nucleus Sampling (p=0.95)
Output 1: Once upon a time in a land far far away,  There lived an  Adventurer who were called by the gods 

| **p** | **Output 1** | **Output 2** | **Output 3** |
|----------|--------------|--------------|--------------|
| 0.97     |Once upon a time in a land far far away, Ȓu I was lucky enough to witness the keys of creation convey the primal spirits and truth of creation. Ȓu was reminded of the absurdity of such crude stargazing when I became contaminated with their false lore about humans, �|Once upon a time in a land far far away, _____ (seventh ed.), written by Phisate of Navias, but scanned and posted by Daniel Schwartz It has been noticed (March 2008) that every 'Title of Person of Notary' either seems excessively long, hesitates to mention | Once upon a time in a land far far away,  Campion's predecessor enjoyed a number of sons and daughters married to Clements Hold El. Learning of their fabulations, Clan Evangeline struck ever harder at their kin - forcing them to shrink and adding even more delights to their nursery,|
| 0.95     |Once upon a time in a land far far away,  There lived an  Adventurer who were called by the gods of a maiden named Mors  When they had many adventures.    One day the Adventurer saw a scene they had never seen before of two warriors that each had a|Once upon a time in a land far far away, _______ lived peacefully. She returned from Sweden once a month. As she wrote a rough version of it to another sister who also had briefly immigrated with her 8th September 1923. The epithet blacks in her body.|Once upon a time in a land far far away, ******** grew two white grapes for his birthday cake…… Although the fact that the space marines were needed under some circumstances made him eat up quite a bit of meat, this dish still had some specialties! ******** wanted his festive treat after all|
| 0.9      |Once upon a time in a land far far away, ichor was something the doctor kept in a pocket, mostly washed down with thick wine. They tried that on Solvanus, who wasn't quite so slender, although he wore it almost daily. On the Fereldan|Once upon a time in a land far far away, urn of urn of red forever returned to the cavern. This urn of red was the bride of Mundus, It carried gold from the Dwemer city of Vivec She and her attendants slept but one night as|Once upon a time in a land far far away, 《Fire bird》 released its power and separated into《Alligator》 and《Albino bird》. In reality, 《Alligator》 was a girl named Asha who was holding an �|
| 0.8      |Once upon a time in a land far far away, _________. You think I'm angry? _________. I don't like it when you're angry. _________. You're mad! _________. You think I'm angry? _________. That's because I think you're angry!|Once upon a time in a land far far away, ikebe wen keepe, ikebe wen come by like a flower. ikebe wen come by, ikebe wen come by like a flower, ikebe wen come by like a flower.|Once upon a time in a land far far away,  a warrior with a bad temper, a stupid cat and a horny attitude set off to track down the chief's daughter, but now the heroine and the rat killer are on the trail of a young man with a violent streak who turns out to be|
| 0.7      |Once upon a time in a land far far away,  Alicia and her mother were cast out of their home for their mistreatment by the King of Demon Land.   The king decreed that they had to live in a prison.   What did they do?   They ran away from home|Once upon a time in a land far far away, ichor was filled with deadly poison. A creature was sealed away, like a secret safe, within a piece of magic that could not be opened. Once open, the creature could never be freed. A soul that survived within the tomb would eventually fade|Once upon a time in a land far far away,  an immortal called the Master resided. He was called by the Great Master of the Gods, "the master of all". His name was Sir Hye, and he ruled the earth with his golden light. It was Sir Hye's intention to|

I noticed the following across outputs:

1. Higher p values tend to increase the diversity and creativity of the output but can lead to degenerative text.

2. Lower p values limit token selection to higher-probability options, producing more focused and better outputs with a lower risk of random symbols or unrelated phrases.



## 2.4 More variations on decoding algorithms (10 pts)

Nucleus sampling was definitely not the end of the road in terms of new decoding algorithms. Even in the past few years, new decoding algorithms have been proposed to address some limitations of existing algorithms.

Two in particular are:
- Typical Sampling ([Meister et al. (2022)](https://arxiv.org/abs/2202.00666))
- Eta Sampling ([Hewitt et al. (2022)](https://arxiv.org/abs/2210.15191))

For this question, CHOOSE ONE of the two algorithms presented above. Below, please describe in a few sentences what your chosen algorithm does in a novel way and the broad motivation behind it. Along with this description, present 3 sampled outputs for the same prompt as above (you can use one hyperparameter value for all of these).



I choose typical Sampling.

This is solution aims to reduce monotony in the text generation from greedy or nucleus sampling and make it more natural as well as human-like, by making the generated sentences likley enough at one moment of time but then switching out for another set of text. While standard methods would cluster around a few high-probability words or treat extremes equally, the typical sampling picks word in accordance with how close it is to expected uncertainty of model predictions.

Instead of using just the most probable word, it aims to pick words that fit well in the context of the sentence, which resembles natural human speech. Because it uses words within a range that fit in average uncertainty, typical sampling helps you toward avoiding repetition and the output sounds more natural language-like.

In [41]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

class LM:
    def __init__(self, model_name="openai-community/gpt2-medium"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.model.eval()

    def typical_sampling(self, prompt: str, eta: float = 0.2, max_length: int = 64) -> str:
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
        otp_ids = input_ids[0].tolist()

        for _ in range(max_length):
            otps = self.model(input_ids=input_ids)
            logits = otps.logits[:, -1, :]
            probs = torch.softmax(logits, dim=-1)
            srtd_probs, srtd_idx = torch.sort(probs, descending=True)
            entropy = -torch.sum(srtd_probs * torch.log(srtd_probs + 1e-9), dim=-1, keepdim=True)
            typical_mask = torch.abs(-torch.log(srtd_probs + 1e-9) - entropy) <= eta
            typical_probs = srtd_probs[typical_mask]
            typical_idx = srtd_idx[typical_mask]

            if typical_probs.numel() > 0:
                nxt_tkn_idx = torch.multinomial(typical_probs, num_samples=1).item()
                nxt_tkn_id = typical_idx[nxt_tkn_idx].item()
            else:
                nxt_tkn_id = srtd_idx[0, 0].item()

            otp_ids.append(nxt_tkn_id)
            input_ids = torch.cat([input_ids, torch.tensor([[nxt_tkn_id]])], dim=-1)

            if nxt_tkn_id == self.tokenizer.eos_token_id:
                break

        return self.tokenizer.decode(otp_ids, skip_special_tokens=True)



In [42]:
prompt = "Once upon a time in a land far far away, "
GPT2LM = LM()

print("Output 1:", GPT2LM.typical_sampling(prompt, eta=0.2))
print("Output 2:", GPT2LM.typical_sampling(prompt, eta=0.2))
print("Output 3:", GPT2LM.typical_sampling(prompt, eta=0.2))

Output 1: Once upon a time in a land far far away, _____ played this ____ music that didn't play with each other but got heard around town: a church bells chorus." —http://www.ibiblio.org/sounds/libre.htm

Another comment says:

"While watching some opera recently, I stumbled across this picture on Pinterest
Output 2: Once upon a time in a land far far away, ichor from all creatures within my presence dried into blood.  There is something terribly, terribly wrong.  What am I, what are we? What will my rebirth do?
Output 3: Once upon a time in a land far far away, 【Herakles】 defeated his son, 【Enraged】 defeated his daughter, 【Mystery】 defeated his daughter. Now he wants to rid this world of them all." 【Vagrant】 thought back to her encounter with them and a sad thought began to escape her heart. It seemed that


| Decoding Method | **Output 1** | **Output 2** | **Output 3** |
|-----------------|--------------|--------------|--------------|
| Typical Sampling               | Once upon a time in a land far far away, _____ played this ____ music that didn't play with each other but got heard around town: a church bells chorus." —http://www.ibiblio.org/sounds/libre.htm Another comment says: "While watching some opera recently, I stumbled across this picture on Pinterest|Once upon a time in a land far far away, ichor from all creatures within my presence dried into blood.  There is something terribly, terribly wrong.  What am I, what are we? What will my rebirth do?|Once upon a time in a land far far away, 【Herakles】 defeated his son, 【Enraged】 defeated his daughter, 【Mystery】 defeated his daughter. Now he wants to rid this world of them all." 【Vagrant】 thought back to her encounter with them and a sad thought began to escape her heart. It seemed that|

## 2.5 BONUS (Up to 15 pts)

Can you find a prompt where the continuations do not differ much across multiple sampling strategies, even when we use high temperatures or high p values? (Hint: Think about overfitting)

When we use different sampling methods like temperature sampling, nucleus sampling, or typical sampling with high randomness (like high temperature or high p-values), we usually expect the model to produce more varied and creative outputs. However, there are cases where these methods still generate similar responses. This usually happens when the model has a strong association with certain prompts.

For example, prompts that ask for common facts or well-known phrases might trigger almost identical answers across sampling methods. The model is so familiar with these prompts that it keeps giving the same response, even when we try to increase randomness.

For example: The 1st president of India was...

# 3. Prompting (50 pts)

In this problem, we will try various prompting approaches and prompt an LLM for a Math Reasoning Benchmark called [GSM8K](https://github.com/openai/grade-school-math), which contains grade school math word problems. This is a very common _reasoning_ benchmark used to test various LLMs.

The LLM that we will be using is [Google Gemini](https://gemini.google.com/). We will be prompting Gemini by using an API call to the Gemini Model. Normally, you can also prompt Open Source LLMs via the HuggingFace Library, however due to compute constraints, we use Gemini in this problem.

## Setting up the GSM8K Dataset and Google Gemini

Follow the steps below to download the GSM8K Dataset and to setup Google Gemini on Colab. You will automatically get points for this subpr

In [43]:
from datasets import load_dataset

dataset = load_dataset("gsm8k", 'main')

In [44]:
len(dataset['train']), len(dataset['test'])

(7473, 1319)

In [45]:
# An example instance of this dataset

dataset['test'][6]

{'question': 'Toulouse has twice as many sheep as Charleston. Charleston has 4 times as many sheep as Seattle. How many sheep do Toulouse, Charleston, and Seattle have together if Seattle has 20 sheep?',
 'answer': 'If Seattle has 20 sheep, Charleston has 4 * 20 sheep = <<20*4=80>>80 sheep\nToulouse has twice as many sheep as Charleston, which is 2 * 80 sheep = <<2*80=160>>160 sheep\nTogether, the three has 20 sheep + 160 sheep + 80 sheep = <<20+160+80=260>>260 sheep\n#### 260'}

### Gemini Setup (from the official [Gemini documentation](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/get-started/python.ipynb))


Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel.

---

Give it the name `GEMINI_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GEMINI_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [46]:
!pip install google-generativeai

In [47]:
# All imports for this question
from google.colab import userdata
import google.generativeai as genai
from datasets import Dataset
import random
from typing import Callable, List, Any

In [48]:
GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)

In [49]:
# Test if your setup is working, do not change the model name
model = genai.GenerativeModel("gemini-1.0-pro")
response = model.generate_content("What is Natural Language Processing? Explain it to a five year old.")
print(response.text)

Imagine you have a special friend named Alexa or Siri who can understand what you say and answer your questions. Natural Language Processing (NLP) is like giving Alexa or Siri superpowers! It's like a secret code that computers use to chat with us in a way that we can understand, just like you chat with your friends. It's like having a translator who helps the computer understand us better. When we talk to the computer, NLP translates our words into a language the computer can understand. It's like a super-smart assistant that helps Alexa and Siri understand us and give us the best answers they can!


## 3.1 Data and Prompting Setup (15 + 5 pts)

In this part, we will create some boilerplate code to process our dataset and generate prompts from the dataset.



### Processing the GSM8K Dataset

In [50]:
def process_gsm8k_answers(dataset: Dataset) -> Dataset:
    """
    Processes the GSM8K dataset to remove reasoning chains and retain only the numerical answers.
    Assumes answers are separated from reasoning by the '###' string.

    Args:
    dataset (Dataset): Huggingface Dataset object for GSM8K.

    Returns:
    Dataset: Processed Dataset object with numerical answers only.
    """

    def extract_answer(sample):
        # IMPLEMENT HERE
        # Split the answer using '###' and return a dictionary with the key 'processed_answer'
        ans = sample['answer'].split('###')
        processed_answer = ans[-1].strip()
        return {'processed_answer': processed_answer}


    return dataset.map(extract_answer)

### Building Prompts (15 pts)

We will be implementing FIVE (5) prompting methods. See their descriptions below -
1. **Zero-Shot Answer Only (2 pts)**: You prompt the model to only generate the answer to the question

2. **Zero-Shot Chain of Thought (CoT) (3 pts)**: Refer to the [Chain of Thought Paper](https://arxiv.org/abs/2201.11903). CoT refers to a reasoning chain that is generated by the model before generating the actual answer. This has shown to improve performance. In this setup, you will prompt the model to generate a reasoning chain before the answer.

3. **5-Shot Answer Only (2 pts)**: You provide some in-context examples to prompt the model with to generate the answer. This is analogous to Approach 1. Use the a random set of 5 examples from the training set to create the in-context examples.

4. **5-Shot CoT (3 pts)**: Combine Approaches 2 and 3 to do 5-shot CoT prompting.

5. **Your own prompt! (5 pts)**: Try something new. Think about how you solve Math problems and implement your own prompting method.

In [51]:
def prompt_generation_zero_shot(problem: str) -> str:
    """
    Zero-shot prompt.

    Returns:
    str: The generated prompt.
    """
    # IMPLEMENT HERE
    return f"Answer this math problem: {problem}"

In [52]:
def prompt_generation_zero_shot_cot(problem: str) -> str:
    """
    Zero-shot Chain of Thought (CoT) prompt.

    Returns:
    str: The generated prompt.
    """
    # IMPLEMENT HERE
    return f"Solve this math problem step-by-step and then provide the final answer:\n\nProblem: {problem}"

In [53]:
def prompt_generation_5_shot(problem: str, training_set: Dataset) -> str:
    """
    5-shot prompt generation for GSM8K problems. Randomly selects 5 examples from the training set.

    Returns:
    str: The generated prompt with 5 in-context_examples.
    """
    # IMPLEMENT HERE
    exs = random.sample(list(training_set), 5)
    in_cntxt_ex = ""

    for example in exs:
      in_cntxt_ex += f"Problem: {example['question']}\nAnswer: {example['processed_answer']}\n\n"

    prompt = f"{in_cntxt_ex}Problem: {problem}\nAnswer:"

    return prompt

In [54]:
def prompt_generation_5_shot_cot(problem: str, training_set: Dataset) -> str:
    """
    5-shot Chain of Thought (CoT) prompt generation. Randomly selects 5 examples
    from the training set and includes reasoning steps.

    Returns:
    str: The generated prompt with 5 CoT in-context examples.
    """
    # IMPLEMENT HERE
    exs = random.sample(list(training_set), 5)
    in_cntxt_ex = ""

    for example in exs:
      in_cntxt_ex += f"Problem: {example['question']}\nSolution: {example['answer']}\n\n"

    prompt = f"{in_cntxt_ex}Problem: {problem}\nSolution:"

    return prompt

In [55]:
# Feel free to change the method definition

def my_prompt(problem: str) -> str:
    """
    Your own unique way of prompting an LLM for Math word problems.

    Returns:
    str: The generated prompt
    """
    # IMPLEMENT HERE
    return (
        f"Please solve the following math problem step-by-step, and double-check your final answer.\n\n"
        f"Problem: {problem}\n"
        f"Solution (with detailed reasoning):"
    )

## 3.2 Prompting Gemini and Implementing Self-Consistency (5 + 5 + 10 pts)

Here, you will help build the wrapper for prompting Gemini using the prompt methods you have designed above.

You will then also implement Self-Consistency based prompting. Refer to the [Self-Consistency Paper](https://arxiv.org/abs/2203.11171). In order to implement Self-Consistency, you generate multiple Zero-Shot CoT (Approach 2 in the prompting methods) candidates, and take a majority vote of the answers predicted by each candidate.

### First, write the function where you will process the answer generated by the model. (5 pts)

Note that answer processing changes for different prompt types, so this function also takes in the name of the method in its argument.

In [56]:
import re

def answer_processing(prediction: str, prompt_function: Any) -> str:
    """
    Processes the model's generated output to extract the final answer.

    Returns:
    str: The processed numerical answer.
    """
    prompt_name = prompt_function.__name__
    new_pred = re.sub(r'[^\d\s.]', '', prediction)
    ans = re.findall(r'\d+\.?\d*', new_pred)

    if ans:
        if prompt_name in ["prompt_generation_zero_shot", "prompt_generation_5_shot"]:
            return ans[-1]
        elif prompt_name in ["prompt_generation_zero_shot_cot", "prompt_generation_5_shot_cot", "my_prompt"]:
            largest_answer = max(map(float, ans))
            return str(int(largest_answer)) if largest_answer.is_integer() else str(largest_answer)
    return ""


In [57]:
# Do not change, method to calculate accuracy from predictions and ground truth labels

def evaluate_accuracy(predictions: List[str], ground_truths: List[str]) -> float:
    correct = 0
    total = len(predictions)

    for pred, true in zip(predictions, ground_truths):
        standardized_true = true.lstrip('# ').strip()
        if pred == standardized_true:
            correct += 1

    accuracy = correct / total
    return accuracy * 100

### Next, write the wrapper function where you use all the building blocks constructed above to prompt the Gemini model (5 + 10 pts)


On how to prompt Gemini, refer to the [Gemini Text Generation Handbook](https://ai.google.dev/gemini-api/docs/text-generation?lang=python).

Hint: Reading this will help you figure out how to generate multiple candidates to implement Self-Consistency.

In [58]:
from collections import Counter
from typing import Callable, List, Any

def prompt_gemini_with_self_consistency(
    problem: str,
    prompt_function: Callable[[str], str],
    model_instance: Any,
    num_candidates: int
) -> str:

    pos_ans = []
    for _ in range(num_candidates):
        if prompt_function in [prompt_generation_5_shot, prompt_generation_5_shot_cot]:
            prompt = prompt_function(problem, training_set=None)
        else:
            prompt = prompt_function(problem)
        res = model_instance.generate_content(prompt)
        ans = answer_processing(res.text.strip(), prompt_function)
        pos_ans.append(ans)
    if pos_ans:
        final_ans = Counter(pos_ans).most_common(1)[0][0]
        return final_ans
    return ""

In [59]:
def pipeline_generate(
    model_instance: Any,
    test_set: Dataset,
    prompt_function: Callable[[str], str],
    process_answer_function: Callable[[str], str],
    evaluation_function: Callable[[List[str], List[str]], float],
    self_consistency: int,
    training_set: Dataset = None
) -> float:
    """
    Args:
    model_instance (Any): The Google Gemini model instance.
    test_set (Dataset): The GSM8K test set to evaluate on.
    prompt_function (Callable): Function to generate prompts for the test set.
    process_answer_function (Callable): Function to process the model's generated answers.
    evaluation_function (Callable): Function to evaluate model's answers against the ground truth.
    self_consistency: Number of samples to run self-consistency approach on.
    If negative, 0 or 1, this implies regular prompting.

    Returns:
    float: The accuracy of the model on the test set.
    """
    preds = []
    ground_truths = []

    for sample in test_set:
        problem = sample['question']
        ground_truth = sample['processed_answer']
        ground_truths.append(ground_truth)

        if self_consistency > 1:
            final_answer = prompt_gemini_with_self_consistency(
                problem, prompt_function, model_instance, num_candidates=self_consistency
            )
        else:
            if prompt_function in [prompt_generation_5_shot, prompt_generation_5_shot_cot]:
                prompt = prompt_function(problem, training_set=training_set)
            else:
                prompt = prompt_function(problem)

            response = model_instance.generate_content(prompt)
            prediction = response.text.strip()
            final_answer = process_answer_function(prediction, prompt_function)
        preds.append(final_answer)

    accuracy = evaluation_function(preds, ground_truths)
    return accuracy


In [60]:
gsm8k_test_processed = process_gsm8k_answers(dataset['test'])
gsm8k_training_set = process_gsm8k_answers(dataset['train'])
# The following line is just to test your systems, comment this line out to report results on the entire
gsm8k_test_processed = Dataset.from_dict(gsm8k_test_processed[:5])

# Run model generation with zero-shot prompt generation
accuracy = pipeline_generate(
    model_instance=model,
    test_set=gsm8k_test_processed,
    prompt_function=prompt_generation_zero_shot_cot, # Change this to test different prompt methods
    process_answer_function=answer_processing,
    evaluation_function=evaluate_accuracy,
    self_consistency=2,
    training_set=gsm8k_training_set
)

print(f"Accuracy: {accuracy}%")

Accuracy: 60.0%


## 3.3 Complete this table based on your implementation in 3.2 and answer the following questions (5 + 5 pts)

### Round each value up to two decimal points (5 pts)

Method|Accuracy
---|---|
0-shot| 80%
0-shot CoT| 60%
5-shot| 80%
5-shot CoT| 20%
My prompt| 40%
0-shot CoT Self-Consistency| 60%

### What was the intuition behind the prompt that you designed? (2 pts)

The intuition behind the prompt was to make the model go through a careful solution step-by-step, prompting the model to think about how it would solve this problem before providing its answer. This essentially tells the model to double-check its original response, which can help minimize error and improve accuracy especially for multi-step problems or complex calculations.

### What are the merits and demerits of using advanced prompting approaches like Chain of Thought or Self-Consistency? (3 pts)

**Merits:**

 1.These COT promptings encourage the model to think through every step in a solution, the reasoning often leads to increased accuracy for more complicated problems.

2. These Self Consistency approach outputs a solution, and you present it with multiple answers, it will pick whichever answer came out as the most common, and this reduces random errors.

3. These methods work well for multi-step or reasoning-heavy tasks, since simple answers may be less accurate.

**Demerits:**

1. They might generate many outputs that can be computationally costly and time-consuming.

2. Such prompting methods usually need a low-level design and tuning of parameters, which can lead to increased complexity in implementation.

3. These methods won’t bring large accuracy benefits to very simple questions, and they are strong for such scenerios.